# 爬取下厨房的数据


数据的获取，是数据分析和数据可视化的前提。

我们通过爬取下厨房网站的菜谱数据，继续进行爬虫技术的学习。

In [1]:
import requests  # 导入request 库
# 获取网页源代码
url = 'https://www.xiachufang.com/category/957/'
res = requests.get(url)
res.encoding='utf-8'
print(res.text)


<!DOCTYPE html>

<!--[if lt IE 7]> <html class="lt-ie9 lt-ie8 lt-ie7 show-anti-ie"> <![endif]-->
<!--[if IE 7]>    <html class="lt-ie9 lt-ie8 show-anti-ie"> <![endif]-->
<!--[if IE 8]>    <html xmlns:ng="http://angularjs.org" class="lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class=""> <!--<![endif]-->

<head>
  


    <title>鱼的家常做法大全_鱼怎么做好吃_菜谱大全_下厨房</title>

    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
    <meta http-equiv="content-type" content="text/html;charset=UTF-8;">
    <meta http-equiv="x-dns-prefetch-control" content="on">
    <link rel="dns-prefetch" href="//s.chuimg.com">
    <meta name="viewport" content="width=device-width">
    <meta name="apple-itunes-app" content="app-id=460979760"/>
    <meta name="baidu-site-verification" content="HNAbHIxmbo" />
    <meta name="renderer" content="webkit">
    <link href="https://s.chuimg.com/favicon.ico" rel="shortcut icon">

    <script src="//dup.baidustatic.com/js/ds.js"></s

接下来，我们对请求回来的文本数据，使用lxml进行解析。

**lxml**是python的一个解析库，支持HTML和XML的解析，支持**XPath**解析方式，而且解析效率非常高。

**XPath**，全称**XML Path Language**，即XML路径语言，它是一门在XML文档中查找信息的语言，它最初是用来搜寻XML文档的，但是它同样适用于HTML文档的搜索

XPath的选择功能十分强大，它提供了非常简明的路径选择表达式，另外，它还提供了超过100个内建函数，用于字符串、数值、时间的匹配以及节点、序列的处理等，几乎所有我们想要定位的节点，都可以用XPath来选择。

In [ ]:
from lxml import etree  # 为了使用xpath解析工具，我们先导入etree
parsed = etree.HTML(res.text)  # 对返回的源代码进行解析，解析后的结果命名为parsed

XPath 的调用语法为：

- `parsed.xpath('your xpath')`

下面来看一个例子

我们首先在浏览器中找到内容的xpath，然后输入下面的代码里面。

In [ ]:
parsed.xpath('/html/body/div[4]/div/div/div[1]/div[1]/div/div[2]/div[2]/div[2]/ul/li[2]/div/div/p[1]/a/text()')

Xpath 是一个路径语言

- `/`单个斜杠是绝对路径

- `//`两个斜杠是相对路径

- `[]`方括号是属性条件
- `@`是选择属性

In [ ]:
parsed.xpath('//p/a[@href="/recipe/103739083/"]/text()')

通过修改最后一个路径，我们还可以获取菜谱的href属性，也就是菜谱编号。

In [ ]:
parsed.xpath('//p/a[@href="/recipe/103739083/"]/@href')

更多的时候，我们关心如何拿到该页面所有的菜谱信息。

这里我们下载一个`XpathHelper`插件,方便后续的分析。

去掉[1]就可以出现所有的菜名

In [ ]:
parsed.xpath('/html/body/div[4]/div/div/div[1]/div[1]/div/div[2]/div[2]/div[2]/ul/li/div/div/p[1]/a/text()')

上面的路径过于冗杂了，我们采用相对路径写法

In [ ]:
parsed.xpath('//div[@class="ing-recipe"]/div[@class="normal-recipe-list"]/ul/li/div/div/p[1]/a/text()')

对于菜谱编号也是一样的

In [ ]:
parsed.xpath('//div[@class="ing-recipe"]/div[@class="normal-recipe-list"]/ul/li/div/div/p[1]/a/@href')

接下来，我们使用DataFrame将其保存起来

In [ ]:
import requests  # 导入request 库
from lxml import etree
res = requests.get('https://www.xiachufang.com/category/957/')
res.encoding='utf-8'
parsed = etree.HTML(res.text)
name = parsed.xpath('/html/body/div[4]/div/div/div[1]/div[1]/div/div[2]/div[2]/div[2]/ul/li/div/div/p[1]/a/text()')
href = parsed.xpath('//div[@class="ing-recipe"]/div[@class="normal-recipe-list"]/ul/li/div/div/p[1]/a/@href')

import pandas as pd 
recipe = pd.DataFrame()
recipe['name'] = name
recipe['href'] = href
recipe
# recipe.to_csv('recipe.csv',index = False,encoding = 'gbk')  # 这一行可以把数据存储到csv文件中

# 实现翻页功能

通过上面的代码，我们已经爬取了一个页面的菜谱信息，接下来我们学习如何翻页     

In [ ]:
res = requests.get(url+'?page='+str(2),headers = headers)
res.text

In [ ]:
import requests  # 导入request 库
import pandas as pd
from lxml import etree
import pandas as pd 
# 获取网页源代码
url = 'https://www.xiachufang.com/category/957/' # 设置url
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}

data_all = pd.DataFrame()
for i in range(1,11+1):
    print('正在爬取第{}页的数据'.format(i))
    res = requests.get(url+'?page='+str(i),headers = headers) # 访问url，获取源代码
    res.encoding='utf-8'  # 解码
    
    parsed = etree.HTML(res.text) # 使用 etree对文本进行解析，得到树型的结构

    # 获取菜名
    name = parsed.xpath('/html/body/div[4]/div/div/div[1]/div[1]/div/div[2]/div[2]/div[2]/ul/li/div/div/p[1]/a/text()')
    # 获取菜的主页链接
    href = parsed.xpath('//div[@class="ing-recipe"]/div[@class="normal-recipe-list"]/ul/li/div/div/p[1]/a/@href')
    
    # 保存数据
    recipe = pd.DataFrame()
    recipe['name'] = name
    recipe['href'] = href
    
    # 将本页的数据，添加到总数据表格中
    data_all = pd.concat([data_all,recipe],ignore_index = True)
data_all
# recipe.to_csv('recipe.csv',index = False,encoding = 'gbk')  # 取消注释这一行，可以将数据保存在自己的电脑

# 评论爬取和异步加载

下面我们来尝试爬取菜谱的评论数据，在其中我们学习**异步加载**的知识。

In [ ]:
import requests  # 导入request 库
from lxml import etree
import pandas as pd
# 指定请求头
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}
# 获取网页源代码
page = 1
data_all = pd.DataFrame()
while True:
    res = requests.get('https://www.xiachufang.com/recipe/100458895/questions/?order=hot&page='+str(page),headers = headers)
    res.encoding='utf-8'
    #print(res.text)
    print('正在爬取第{}页的数据'.format(page))
    
    parsed = etree.HTML(res.text)

    nickname = parsed.xpath('//div[@class="right-top info"]/a/text()') # 用户名
    href =  parsed.xpath('//div[@class="right-top info"]/a/@href')  # 主页链接
    
    comment = parsed.xpath('//div[@class="right-bottom"]')  # 评论内容
    comment = [com.text.split()[0] for com in comment]   # 对评论内容进行处理
    
    time =  parsed.xpath('//div[@class="right-top info"]/span')  # 评论时间
    time = [t.text.split()[0] for t in time]   # 对评论时间进行处理
    vote = parsed.xpath('//div[@class="right-top info"]/span/div/span/span/text()')  # 点赞数
    

    # 保存数据
    data = pd.DataFrame()
    data['nickname'] = nickname
    data['href'] = href
    data['comment'] = comment
    data['time'] = time
    data['vote'] = vote
    

    # 汇入总表格
    data_all = pd.concat([data_all,data],   ignore_index = False)


    
    # 爬虫结束条件
    if res.text.find('暂无最热的留言') == -1:
        page = page +1
    else:
        break 
data_all
# recipe.to_csv('recipe.csv',index = False,encoding = 'gbk')